In [2]:
sc

StatementMeta(SmallPool, 25, 3, Finished, Available, Finished)

<SparkContext master=yarn appName=Transformation_4_String_manipulation_and_sorting_SmallPool_1727183303>

Run the 'mount_configuration' notebook to access the mount

In [3]:
mssparkutils.notebook.run('mount_configuration')

StatementMeta(SmallPool, 25, 4, Finished, Available, Finished)

'Mounting'

Run the mounting notebook to access the mount point

In [4]:
job_id = mssparkutils.env.getJobId()
mount_point = 'synfs:/notebook/'+job_id+'/lake'

StatementMeta(SmallPool, 25, 5, Finished, Available, Finished)

In [5]:
print(mount_point)

StatementMeta(SmallPool, 25, 6, Finished, Available, Finished)

synfs:/notebook/25/lake


Reading the source file

In [6]:
df = spark.read.parquet(mount_point+'/JoinedData/*.parquet')

StatementMeta(SmallPool, 25, 7, Finished, Available, Finished)

In [7]:
display(df)

StatementMeta(SmallPool, 25, 8, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 4260d34a-e4b9-44cb-8e6a-8885595bbc81)

Storing the names of all the columns in a variable

In [8]:
allColumns = df.columns

StatementMeta(SmallPool, 25, 9, Finished, Available, Finished)

In [9]:
print(allColumns)

StatementMeta(SmallPool, 25, 10, Finished, Available, Finished)

['Education Level', 'Line Number', 'Year', 'Month', 'State', 'Labor Force', 'Employed', 'Unemployed', 'Industry', 'Gender', 'Date Inserted', 'Aggregation Level', 'Data Accuracy', 'UnEmployed Rate Percentage', 'Expected Salary Range in USD']


Replacing the space in the column names with an underscore('_')

In [10]:
for column in allColumns:
    df = df.withColumnRenamed(column,column.replace(' ','_'))

StatementMeta(SmallPool, 25, 11, Finished, Available, Finished)

In [11]:
display(df)

StatementMeta(SmallPool, 25, 12, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, fe1e7944-0d56-4b00-b84e-34851efd50d1)

The data in 'Expected_Salary_Range_in_USD' is inconsistent

In [12]:
df.select('Expected_Salary_Range_in_USD').show()

StatementMeta(SmallPool, 25, 13, Finished, Available, Finished)

+----------------------------+
|Expected_Salary_Range_in_USD|
+----------------------------+
|              80000 - 120000|
|               50000 - 70000|
|               60000 - 90000|
|               50000 - 70000|
|             100000 - 150000|
|               60000 - 90000|
|              80000 - 120000|
|               50000 - 70000|
|              80000 - 120000|
|               50000 - 70000|
|             100000 - 150000|
|               60000 - 90000|
|              80000 - 120000|
|               50000 - 70000|
|               50000 - 70000|
|              80000 - 120000|
|             100000 - 150000|
|               60000 - 90000|
|               50000 - 70000|
|              80000 - 120000|
+----------------------------+
only showing top 20 rows



Split the "Expected_Salary_Range_in_USD" using into two columns "-"

Extracting two columns into two dataframes

In [13]:
from pyspark.sql.functions import *
df_split1 = df.withColumn('Min_Salary_USD',split("Expected_Salary_Range_in_USD"," - ")[0].cast('integer'))

StatementMeta(SmallPool, 25, 14, Finished, Available, Finished)

In [14]:
display(df_split1)

StatementMeta(SmallPool, 25, 15, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 88216985-966b-418b-81a6-9f892de32361)

In [15]:
df_split2 = df_split1.withColumn('Max_Salary_USD',split('Expected_Salary_Range_in_USD'," - ")[1].cast('integer'))

StatementMeta(SmallPool, 25, 16, Finished, Available, Finished)

In [16]:
display(df_split2)

StatementMeta(SmallPool, 25, 17, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 1f032464-575b-4d5b-a9c8-d388e46f1a23)

Drop the inconsistent data

In [17]:
df_dropped = df_split2.drop('Expected_Salary_Range_in_USD')

StatementMeta(SmallPool, 25, 18, Finished, Available, Finished)

In [18]:
display(df_dropped)

StatementMeta(SmallPool, 25, 19, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, f7ffb57c-de2b-493c-8aa5-e9b086739451)

Sort by 'Date_Inserted' and 'UnEmployed_Rate_Percentage' column

In [23]:
df_ordered = df_dropped.orderBy(desc('Date_Inserted'),desc('UnEmployed_Rate_Percentage'))

StatementMeta(SmallPool, 25, 24, Finished, Available, Finished)

In [25]:
display(df_ordered)

StatementMeta(SmallPool, 25, 26, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, ea2d8e2c-e783-42b8-a888-1b49831b7bc1)

Write the transformed data into a location

In [24]:
df_ordered.write.format('parquet')\
                .mode('overwrite')\
                .save(mount_point+'/DataSorted')

StatementMeta(SmallPool, 25, 25, Finished, Available, Finished)